# 01 - Exploração e Limpeza de Dados
Este notebook realiza a **análise exploratória dos dados (EDA)** e o **pré-processamento inicial** do dataset *Air Quality and Health Impact*.  
As etapas incluem:

1. Carregamento e inspeção do dataset bruto  
    - Remoção de colunas não utilizáveis  
    - Separação entre variáveis preditoras e alvo  
2. Análise exploratória (EDA)
   - Verificação de tipos, estatísticas, duplicatas e ausentes
   - Histogramas
   - Boxplots
   - Correlação
   - Investigação de outliers
3. Normalização e salvamento do dataset
   - Salvamento do scaler com `StandardScaler`
   - Salvamento do dataset processado

In [ ]:
# Imports das funções utilitárias
from src.preprocessing import (load_raw_dataset, remove_columns, split_features_target, plot_histograms, plot_target_distribution, 
                               plot_boxplots, plot_correlation_matrix,find_outliers_iqr , fit_and_save_scaler, save_processed_dataset)

## 1. Carregamento e inspeção do dataset bruto

In [ ]:
df = load_raw_dataset("../data/raw/air_quality_health_impact_data.csv")
print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
df.head()

### Sobre o dataset

Este dataset contém 5.811 registros relacionados a qualidade do ar, condições climáticas e impactos na saúde.  
Possui três áreas principais:

- **Air Quality Metrics**
- **Weather Conditions**
- **Health Impact Metrics**

Uma coluna com o Id:

- `RecordId`

E duas colunas relacionadas ao alvo:

- `HealthImpactScore` → variável contínua (0–100)
- `HealthImpactClass` → variável categórica (classes 0–4)

### Remoção do Score e do ID
A variável `HealthImpactScore` é usada para construir diretamente o valor da classe (`HealthImpactClass`).  
Portanto, usá-la como feature criaria *data leakage*: o modelo teria acesso a informação que ele só deveria ver após prever.

Já o `RecordId`, é apenas um identificador sequencial que não contém informação preditiva.

In [ ]:
df = remove_columns(df, cols=['HealthImpactScore', 'RecordID'])
df.head()

### Conversão de dados

Além de tudo, percebe-se que o alvo, classes, está como ``float64``. Porém, a mesma é categórica, por isso, será feito o `cast` para `int64`

In [ ]:
df["HealthImpactClass"] = df["HealthImpactClass"].astype("int64")

### Separação entre features e alvo

Separar as variáveis preditoras da variável alvo é útil na fase de exploração, pois:

- evita misturar os atributos de entrada com a classe alvo;
- facilita a visualização (histogramas, boxplots, correlação);
- permite identificar outliers e distribuições apenas nas features;
- prepara a base para o pré-processamento e para a modelagem futura.

É uma separação é apenas organizacional.

In [ ]:
feature_df, target = split_features_target(df, target='HealthImpactClass')

## 2. Análise exploratória (EDA)

### Ausência de valores ausentes e duplicados

O dataset não possui valores ausentes nem duplicados.  
Portanto, não será necessário imputação, tampouco remover ou agrupar instâncias repetidas

In [ ]:
# Dados faltosos e duplicados
print("\nLinhas duplicadas:", df.duplicated().sum())

print("\nValores ausentes:")
display(feature_df.isnull().sum())

### Tipos de dados

Todas as variáveis do dataset são numéricas.  
A maior parte das features é composta por valores `float`, exceto três colunas de métricas de saúde:

- `RespiratoryCases`
- `CardiovascularCases`
- `HospitalAdmissions`

Essas três variáveis são do tipo inteiro (`int`) e representam contagens de eventos de saúde.  
Como é comum em dados de contagem, elas apresentam valores extremos (outliers) devido à natureza dos eventos que registram.

In [ ]:
print("Tipos de dados:")
print(feature_df.dtypes)

### Histogramas das features

Os histogramas permitem visualizar a distribuição das variáveis numéricas.  
As variáveis contínuas (`float`) apresentam distribuições suaves e regulares. As variáveis de contagem mostram maior dispersão, o que é esperado para dados epidemiológicos de contagem.

In [ ]:
plot_histograms(feature_df)

### Distribuição da variável alvo

Para a variável alvo `HealthImpactClass`, utilizamos um gráfico de barras (countplot), pois ela é categórica.  
Observa-se um forte desbalanceamento, com predominância da classe 0 (“Very High”).  
Esse desbalanceamento será considerado na fase de modelagem.

In [ ]:
plot_target_distribution(target)

Onde:
- 0: ``Very High`` (HealthImpactScore >= 80)
- 1: ``High`` (60 <= HealthImpactScore < 80)
- 2: ``Moderate`` (40 <= HealthImpactScore < 60)
- 3: ``Low`` (20 <= HealthImpactScore < 40)
- 4: ``Very Low`` (HealthImpactScore < 20)

### Boxplots e análise de outliers

Os boxplots revelam a presença de outliers principalmente nas variáveis de contagem (`RespiratoryCases`, `CardiovascularCases`, `HospitalAdmissions`).  
Esses valores extremos são esperados e representam eventos reais, como picos de casos ou hospitalizações.  
Por isso, optou-se por manter os outliers no dataset.


In [ ]:
plot_boxplots(feature_df)

### Análise de Outliers (IQR)

Foi aplicado a técnica do Intervalo Interquartil (IQR) para identificar valores extremos nas variáveis numéricas.  
Os resultados confirmam que as variáveis de contagem — *RespiratoryCases*, *CardiovascularCases* e *HospitalAdmissions* — apresentam um grande número de outliers.

Isso é esperado, pois essas variáveis representam eventos epidemiológicos reais, que naturalmente podem apresentar picos abruptos devido a surtos localizados, sazonalidade ou regiões com maior densidade populacional.

Os limites inferiores (Q1 - 1.5·IQR) ficaram negativos em alguns casos, o que é matematicamente possível, mas não afeta a interpretação, pois os dados são contagens ≥ 0.  
Portanto, os outliers relevantes são apenas os superiores.

Decidiu-se **manter esses outliers**, pois eles carregam informação importante e refletem fenômenos reais, sendo essenciais para uma modelagem fiel.


In [ ]:
for col in ["RespiratoryCases", "CardiovascularCases", "HospitalAdmissions"]:
    outliers, lower, upper = find_outliers_iqr(df[col])
    print(f"\n--- {col} ---")
    print(f"Limite inferior: {lower}")
    print(f"Limite superior: {upper}")
    print(f"Quantidade de outliers: {outliers.count()}")

### Matriz de Correlação

A matriz de correlação mostra que as variáveis apresentam correlações muito baixas entre si.  
Fora a diagonal principal (correlação de uma variável consigo mesma), quase todos os coeficientes estão muito próximos de zero.

Isso indica que: as features são praticamente independentes umas das outras, logo, não há multicolinearidade significativa.
Note que, nenhuma variável apresenta relação linear forte com outra (nenhuma correlação > 0.2).

Portanto, cada variável traz informação relativamente distinta para os algoritmos de classificação. Isso é interessante para alguns modelos como o Naive Bayes, que presume idepedências entre as features.

In [ ]:
plot_correlation_matrix(feature_df)

## 3. Normalização e salvamento do dataset

### Normalização dos Dados

A normalização é uma etapa importante quando utilizamos modelos que dependem de distâncias ou assumem variáveis escaladas de forma semelhante.
Optou-se por utilizar o **StandardScaler**, que transforma cada feature para média 0 e desvio padrão 1.

Essa escolha é adequada para modelos que serão utilizados, como:

- KNN (baseado em distância)
- Regressão Logística (otimização mais estável)
- MLP (rede neural converge mais rápido)
- Naive Bayes Gaussiano (assume que seja uma distribuição normal)

A normalização é ajustada apenas nos **dados de treino**, dentro do procedimento de validação cruzada, para evitar **data leakage**.
Aqui ajustamos (`fit`) o scaler apenas para salvá-lo e disponibilizá-lo para a etapa de modelagem, para um posssível teste de baseline. Além de salvar os transformers, uma boa prática em ML/AI.

Outras normalizações existem como o ``Min-Max``, scaling que transforma os valores para o intervalo [0, 1].  
Apesar de ser útil em alguns casos, ele é muito sensível a outliers — e este dataset possui outliers nas variáveis de contagem (casos e hospitalizações).

O StandardScaler é mais robusto para esse cenário, pois, não "colapsa" valores por causa de outliers e dá uma escala bem distribuída para variáveis que podem assumir valores amplos.

Por isso, o StandardScaler foi escolhido.

## Salvando o dataset processado

Após normalização e ajustes, salvamos:

- o dataset processado (`processed_dataset.csv`)
- o scaler treinado (`preprocessor.joblib`)

Esses arquivos serão usados pelos notebooks de modelagem e avaliação.


In [ ]:
import os
# Caminhos
processed_dir = "../data/processed"
scaler_path = os.path.join(processed_dir, "preprocessor.joblib")
processed_csv_path = os.path.join(processed_dir, "air_quality_health_impact_processed.csv")

# Salvando os dados
# scaler = fit_and_save_scaler(feature_df, save_path=scaler_path)
# save_processed_dataset(df, processed_csv_path)